In [173]:
import numpy as np
from tensorflow import keras
import cv2
import matplotlib.pyplot as plt
import pandas as pd


In [238]:
data = pd.read_csv('full_df.csv')
data

ID  Patient Age Patient Sex    Left-Fundus    Right-Fundus  \
0        0           69      Female     0_left.jpg     0_right.jpg   
1        1           57        Male     1_left.jpg     1_right.jpg   
2        2           42        Male     2_left.jpg     2_right.jpg   
3        4           53        Male     4_left.jpg     4_right.jpg   
4        5           50      Female     5_left.jpg     5_right.jpg   
...    ...          ...         ...            ...             ...   
6387  4686           63        Male  4686_left.jpg  4686_right.jpg   
6388  4688           42        Male  4688_left.jpg  4688_right.jpg   
6389  4689           54        Male  4689_left.jpg  4689_right.jpg   
6390  4690           57        Male  4690_left.jpg  4690_right.jpg   
6391  4784           58        Male  4784_left.jpg  4784_right.jpg   

                               Left-Diagnostic Keywords  \
0                                              cataract   
1                                         normal fundus   
2     laser spot，moderate non proliferative retinopathy   
3                           macular epiretinal membrane   
4                moderate non proliferative retinopathy   
...                                                 ...   
6387                severe nonproliferative retinopathy   
6388             moderate non proliferative retinopathy   
6389                  mild nonproliferative retinopathy   
6390                  mild nonproliferative retinopathy   
6391  hypertensive retinopathy，age-related macular d...   

                              Right-Diagnostic Keywords  N  D  G  C  A  H  M  \
0                                         normal fundus  0  0  0  1  0  0  0   
1                                         normal fundus  1  0  0  0  0  0  0   
2                moderate non proliferative retinopathy  0  1  0  0  0  0  0   
3                     mild nonproliferative retinopathy  0  1  0  0  0  0  0   
4                moderate non proliferative retinopathy  0  1  0  0  0  0  0   
...                                                 ... .. .. .. .. .. .. ..   
6387                 proliferative diabetic retinopathy  0  1  0  0  0  0  0   
6388             moderate non proliferative retinopathy  0  1  0  0  0  0  0   
6389                                      normal fundus  0  1  0  0  0  0  0   
6390                  mild nonproliferative retinopathy  0  1  0  0  0  0  0   
6391  hypertensive retinopathy，age-related macular d...  0  0  0  0  1  1  0   

      O                                           filepath labels  \
0     0  ../input/ocular-disease-recognition-odir5k/ODI...  ['N']   
1     0  ../input/ocular-disease-recognition-odir5k/ODI...  ['N']   
2     1  ../input/ocular-disease-recognition-odir5k/ODI...  ['D']   
3     1  ../input/ocular-disease-recognition-odir5k/ODI...  ['D']   
4     0  ../input/ocular-disease-recognition-odir5k/ODI...  ['D']   
...  ..                                                ...    ...   
6387  0  ../input/ocular-disease-recognition-odir5k/ODI...  ['D']   
6388  0  ../input/ocular-disease-recognition-odir5k/ODI...  ['D']   
6389  0  ../input/ocular-disease-recognition-odir5k/ODI...  ['D']   
6390  0  ../input/ocular-disease-recognition-odir5k/ODI...  ['D']   
6391  0  ../input/ocular-disease-recognition-odir5k/ODI...  ['H']   

                        target       filename  
0     [1, 0, 0, 0, 0, 0, 0, 0]    0_right.jpg  
1     [1, 0, 0, 0, 0, 0, 0, 0]    1_right.jpg  
2     [0, 1, 0, 0, 0, 0, 0, 0]    2_right.jpg  
3     [0, 1, 0, 0, 0, 0, 0, 0]    4_right.jpg  
4     [0, 1, 0, 0, 0, 0, 0, 0]    5_right.jpg  
...                        ...            ...  
6387  [0, 1, 0, 0, 0, 0, 0, 0]  4686_left.jpg  
6388  [0, 1, 0, 0, 0, 0, 0, 0]  4688_left.jpg  
6389  [0, 1, 0, 0, 0, 0, 0, 0]  4689_left.jpg  
6390  [0, 1, 0, 0, 0, 0, 0, 0]  4690_left.jpg  
6391  [0, 0, 0, 0, 0, 1, 0, 0]  4784_left.jpg  

[6392 rows x 19 columns]

In [81]:
left_keywords = data['Left-Diagnostic Keywords']

In [82]:
left_keywords

0                                                cataract
1                                           normal fundus
2       laser spot，moderate non proliferative retinopathy
3                             macular epiretinal membrane
4                  moderate non proliferative retinopathy
                              ...                        
6387                  severe nonproliferative retinopathy
6388               moderate non proliferative retinopathy
6389                    mild nonproliferative retinopathy
6390                    mild nonproliferative retinopathy
6391    hypertensive retinopathy，age-related macular d...
Name: Left-Diagnostic Keywords, Length: 6392, dtype: object

In [158]:
left_keywords[2][10]
x = left_keywords[2][10] + '|,'
keywordsArray = left_keywords[2].split(x)

In [159]:
keywordsArray2 = left_keywords.str.split(x)


In [160]:
keywordsArray2

0                                              [cataract]
1                                         [normal fundus]
2       [laser spot, moderate non proliferative retino...
3                           [macular epiretinal membrane]
4                [moderate non proliferative retinopathy]
                              ...                        
6387                [severe nonproliferative retinopathy]
6388             [moderate non proliferative retinopathy]
6389                  [mild nonproliferative retinopathy]
6390                  [mild nonproliferative retinopathy]
6391    [hypertensive retinopathy, age-related macular...
Name: Left-Diagnostic Keywords, Length: 6392, dtype: object

In [228]:
allKeywords = np.concatenate(keywordsArray2)

allKeywords_df = pd.DataFrame(allKeywords)
num_conditions = allKeywords_df[0].value_counts()
num_conditions[0:11]

normal fundus                             2901
moderate non proliferative retinopathy     915
mild nonproliferative retinopathy          523
cataract                                   304
glaucoma                                   282
pathological myopia                        216
dry age-related macular degeneration       212
hypertensive retinopathy                   191
epiretinal membrane                        175
drusen                                     165
macular epiretinal membrane                162
Name: 0, dtype: int64

In [236]:
classes = {'labels':\
           ['normal',\
            'moderate non proliferative retinopathy',\
            'mild nonproliferative retinopathy',
           'glaucoma',\
            'pathological myopia',\
            'dry age-related macular degeneration',\
            'hypertensive retinopathy',\
            'epiretinal membrane',\
            'drusen',\
            'macular epiretinal membrane',
            'other'],\
            'vector': [[1,0,0,0,0,0,0,0,0,0,0],\
                    [0,1,0,0,0,0,0,0,0,0,0],\
                    [0,0,1,0,0,0,0,0,0,0,0],\
                    [0,0,0,1,0,0,0,0,0,0,0],\
                    [0,0,0,0,1,0,0,0,0,0,0],\
                    [0,0,0,0,0,1,0,0,0,0,0],\
                    [0,0,0,0,0,0,1,0,0,0,0],\
                    [0,0,0,0,0,0,0,1,0,0,0],\
                    [0,0,0,0,0,0,0,0,1,0,0],\
                    [0,0,0,0,0,0,0,0,0,1,0],\
                    [0,0,0,0,0,0,0,0,0,0,1]]}

In [237]:
classes_df = pd.DataFrame(classes)
classes_df

labels                             vector
0                                   normal  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1   moderate non proliferative retinopathy  [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
2        mild nonproliferative retinopathy  [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
3                                 glaucoma  [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
4                      pathological myopia  [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
5     dry age-related macular degeneration  [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
6                 hypertensive retinopathy  [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
7                      epiretinal membrane  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
8                                   drusen  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
9              macular epiretinal membrane  [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
10                                   other  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]

In [ ]:
#loop through data and vectorize the labels 

for 